# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,al lagowa,11.4000,29.1333,30.65,14,19,2.59,SD,1702222246
1,1,tazacorte,28.6290,-17.9293,22.54,73,0,2.06,ES,1702222246
2,2,camocim,-2.9022,-40.8411,30.46,60,92,7.57,BR,1702222246
3,3,kailua-kona,19.6406,-155.9956,23.66,74,0,1.79,US,1702221977
4,4,dolyna,48.9733,24.0094,-3.27,94,97,1.48,UA,1702222246


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
cities_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.6
)

# Display the map plot
cities_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 21)]
ideal_city_data_df = ideal_city_data_df.loc[(ideal_city_data_df["Wind Speed"] < 4.5)]
ideal_city_data_df = ideal_city_data_df.loc[(ideal_city_data_df["Cloudiness"] == 0)]

# Drop any rows with null values
ideal_city_data_df = ideal_city_data_df.dropna()

# Display sample data
ideal_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
1,1,tazacorte,28.6290,-17.9293,22.54,73,0,2.06,ES,1702222246
3,3,kailua-kona,19.6406,-155.9956,23.66,74,0,1.79,US,1702221977
10,10,sur,22.5667,59.5289,24.15,50,0,2.80,OM,1702222247
122,122,sharjah,25.3573,55.4033,25.99,41,0,3.60,AE,1702222256
129,129,turbat,26.0023,63.0440,23.62,16,0,2.35,PK,1702222256
229,229,thayetmyo,19.3167,95.1833,23.44,79,0,0.84,MM,1702222270
255,255,chhor,25.5167,69.7667,22.57,19,0,3.04,PK,1702222275
280,280,haiku-pauwela,20.9219,-156.3051,23.91,81,0,3.58,US,1702222280
390,390,salalah,17.0151,54.0924,26.05,53,0,1.54,OM,1702222054
456,456,karachi,24.9056,67.0822,22.90,68,0,3.60,PK,1702222249


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_data_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
1,tazacorte,ES,28.6290,-17.9293,73,
3,kailua-kona,US,19.6406,-155.9956,74,
10,sur,OM,22.5667,59.5289,50,
122,sharjah,AE,25.3573,55.4033,41,
129,turbat,PK,26.0023,63.0440,16,
229,thayetmyo,MM,19.3167,95.1833,79,
255,chhor,PK,25.5167,69.7667,19,
280,haiku-pauwela,US,20.9219,-156.3051,81,
390,salalah,OM,17.0151,54.0924,53,
456,karachi,PK,24.9056,67.0822,68,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "limit": 20,
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df



Starting hotel search
tazacorte - nearest hotel: App Leyma
kailua-kona - nearest hotel: Kona Seaside Hotel
sur - nearest hotel: Sur Hotel
sharjah - nearest hotel: shakeel bhai room
turbat - nearest hotel: No hotel found
thayetmyo - nearest hotel: No hotel found
chhor - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
salalah - nearest hotel: Muscat International Hotel
karachi - nearest hotel: Quetta Agha Hotel
tinajo - nearest hotel: Villas Tinajo


,City,Country,Lat,Lng,Humidity,Hotel Name
1,tazacorte,ES,28.6290,-17.9293,73,App Leyma
3,kailua-kona,US,19.6406,-155.9956,74,Kona Seaside Hotel
10,sur,OM,22.5667,59.5289,50,Sur Hotel
122,sharjah,AE,25.3573,55.4033,41,shakeel bhai room
129,turbat,PK,26.0023,63.0440,16,No hotel found
229,thayetmyo,MM,19.3167,95.1833,79,No hotel found
255,chhor,PK,25.5167,69.7667,19,No hotel found
280,haiku-pauwela,US,20.9219,-156.3051,81,Inn At Mama's Fish House
390,salalah,OM,17.0151,54.0924,53,Muscat International Hotel
456,karachi,PK,24.9056,67.0822,68,Quetta Agha Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [7]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    color = "City",
    alpha = 0.6,
    hover_cols = ["Hotel Name","Country"]
)

# Display the map plot
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)